# RAG System for Survey Variable Resolution

This Jupyter Notebook is designed to implement a RAG (Retrieval-Augmented Generation) system. The primary purpose of this system is to resolve user-submitted survey variable data and find other variables that are likely to match.

## Overview

The RAG system leverages the power of retrieval-based and generative models for machine learning. It uses a two-step process:

1. **Retrieval**: The system retrieves relevant documents (in this case, survey variables) from a Pinecone DB knowledge source based on the user-submitted data.

2. **Generation**: The system then uses the retrieved documents to generate a response using a Cohere Reranking LLM and OpenAI's ChatOpenAI GPT 3.5 model. This response includes other variables that have a high likelihood of matching the user-submitted data.

## Usage

To use this notebook, input your survey variable data using the `input_data` directory. The RAG system will process this data, retrieve relevant variables from the knowledge source, and generate a list of variables that are likely to match your input.

## Benefits

The RAG system provides a powerful tool for survey data analysis. It can help identify patterns and correlations in the data, which can be invaluable for research and decision-making.

Please note that the accuracy of the system's output depends on the quality and comprehensiveness of the knowledge source. Therefore, it's crucial to continually update and expand the knowledge source to improve the system's performance.

## Setup

In [1]:
import os
import json
import pandas as pd
from langchain.docstore.document import Document
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.retrievers.multi_query import MultiQueryRetriever

import cohere
from pinecone import Pinecone
from pinecone import ServerlessSpec



from getpass import getpass
# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
import os
from getpass import getpass
# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings

In [3]:
import sys
sys.path.insert(0, './prompt_versioning')

from prompt_versions import get_prompt
PROMPT_LTS_VERSION = get_prompt()

#### API KEYS

In [4]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY') or getpass("OpenAI API Key: ")
COHERE_API_KEY = os.getenv('COHERE_API_KEY') or getpass("COHERE API Key: ")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY") or getpass("Enter your Pinecone API key: ")



#### Configuration Values

In [5]:

""""
--- MODEL CONFIG ---
"""

PINECONE_INDEX = "biolincc-labels-001" # Name of the Pinecone index
TEMPERATURE = 0
VECTOR_TEXT_FIELD = "text"
EMBEDDING_MODEL = "text-embedding-ada-002"

""""
--- USER SUBMITTED VARIABLE CONFIG ---
"""
USER_SUBMITTED_VARIABLE_COL = "Variable"
USER_SUBMITTED_LABEL_COL = "Label"
# USER_SUBMITTED_VARIABLE_COL = "name"
# USER_SUBMITTED_LABEL_COL = "description"

#### Logging

In [6]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

### Embedding and Vector DB Setup

In [7]:
pc = Pinecone(api_key=PINECONE_API_KEY)
spec = ServerlessSpec(
    cloud="aws", region="us-west-2"
)

In [8]:
import time

index_name = PINECONE_INDEX
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if index already exists (it shouldn't if this is first time)
if index_name not in existing_indexes:
    # if does not exist, create index
  raise Exception("Pinecone index does not exist")

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 259880}},
 'total_vector_count': 259880}

In [9]:
embed = OpenAIEmbeddings(
    model=EMBEDDING_MODEL, openai_api_key=OPENAI_API_KEY, disallowed_special=()
)

In [10]:
from langchain.vectorstores import Pinecone

vectorstore = Pinecone(index, embed.embed_query, VECTOR_TEXT_FIELD)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_community/vectorstores/pinecone.py:75: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


## Parse User Input

In [11]:
# Specify the path to your CSV file
csv_file_path = 'input_data/ozzys_gen3_test.tsv'

df = pd.read_csv(csv_file_path, sep='\t')

In [12]:
df.describe()

,Variable,Label
count,104,104
unique,104,102
top,submitter_id,Measurement of the mass concentration (mcnc) o...
freq,1,2


## Setup Retriever


**Generation Prompt**
```
    You are a data curator whos role is to harmonize biological variables in the NHLBI (National Heart Lung and Blood) data repository. 
    You are tasked with evaluating input variables from data dictionary that describes new data that will be added to the existing pool of variables in the repository.
    For each new variable, a vector search engine has returned the three nearest existing variables found in the data repository. 
    Your job is determine which of these existing variables, if any, is the best fit for harmonizing the new variable to the existing variable given your understanding of the underlying scientific principles.
    You must rationalize why the selected existing variable was chosen over the others, provide context to the relevancy of each existing variable to the new variable.
    Then you are to provide the user with as much information as we can on how they can align their new variable with the selected existing variable.
    
    When there is no obvious match provide additional context for why you can't make a determination.
    
    Contexts:
    {contexts}

    Original question: {query}
```

In [13]:
from typing import List
from langchain.chains import LLMChain
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser

In [14]:
# Output parser will split the LLM result into a list of queries
class LineList(BaseModel):
    # "lines" is the key (attribute name) of the parsed output
    lines: List[str] = Field(description="Lines of text")


class LineListOutputParser(PydanticOutputParser):
    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = text.strip().split("\n")
        return LineList(lines=lines)

output_parser = LineListOutputParser()


### Prompt Templates

#### Output Generation Prompt

In [15]:
# Define the prompt template
# template = """
#     You are a data curator whos role is to harmonize biological variables in the NHLBI (National Heart Lung and Blood) data repository. 
#     You are tasked with evaluating input variables from data dictionary that describes new data that will be added to the existing pool of variables in the repository.
#     For each new variable, a vector search engine has returned the three nearest existing variables found in the data repository. 
#     Your job is determine which of these existing variables, if any, is the best fit for harmonizing the new variable to the existing variable given your understanding of the underlying scientific principles.
#     You must rationalize why the selected existing variable was chosen over the others, provide context to the relevancy of each existing variable to the new variable.
#     Then you are to provide the user with as much information as we can on how they can align their new variable with the selected existing variable.
    
#     When there is no obvious match provide additional context for why you can't make a determination.
    
#     Contexts:
#     {contexts}

#     Original question: {query}
# """
# print(PROMPT_LTS_VERSION)

GEN_PROMPT = PromptTemplate(
    input_variables=["query", "contexts"],
    template=PROMPT_LTS_VERSION,
)
llm = ChatOpenAI(temperature=TEMPERATURE, openai_api_key=OPENAI_API_KEY)

# Chain
gen_chain = LLMChain(llm=llm, prompt=GEN_PROMPT, output_parser=output_parser)

#### Query Prompt Placeholder

In [16]:
### IS THIS NEEDED? TBD
# query_template = """
#     You are a data curator whos role is to harmonize biological variables in the NHLBI (National Heart Lung and Blood) data repository. 
#     You are tasked with evaluating input variables from data dictionary that describes new data that will be added to the existing pool of variables in the repository.
    
#     Original query: {query}
# """

# QUERY_PROMPT = PromptTemplate(
#     # input_variables=["question"],
#     input_variables=["query"],
#     template=query_template,
# )
# llm = ChatOpenAI(temperature=TEMPERATURE, openai_api_key=OPENAI_API_KEY)

# # Chain
# llm_chain = LLMChain(llm=llm, prompt=QUERY_PROMPT, output_parser=output_parser)

#### Cohere Rerank Retriever

In [17]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
from langchain_community.llms import Cohere

In [18]:
cohere_rerank = CohereRerank(cohere_api_key=COHERE_API_KEY)
retriever=ContextualCompressionRetriever(
    base_compressor=cohere_rerank, 
    base_retriever=vectorstore.as_retriever() 
)


## Connecting RAG elements

In [19]:
from langchain.chains import TransformChain
from langchain.chains import SequentialChain

In [20]:
def retrieval_transform(inputs: dict) -> dict:
    docs = retriever.get_relevant_documents(query=inputs["question"])
    metadata = pd.concat(
        [
            pd.DataFrame(
                [d.metadata for d in docs], columns=["uid", "relevance_score"]
            ),
            pd.DataFrame([d.page_content for d in docs], columns=["matched_text"]),
        ],  axis=1
    )
    docs = [d.page_content for d in docs]
    docs_dict = {
        "query": inputs["question"],
        "contexts": "\n---\n".join(docs),
        "metadata": metadata,
    }
    return docs_dict


retrieval_chain = TransformChain(
    input_variables=["question"],
    output_variables=["query", "contexts", "metadata"],
    transform=retrieval_transform,
)

In [21]:

rag_chain = SequentialChain(
    chains=[retrieval_chain, gen_chain],
    input_variables=["question"],  # we need to name differently to output "query"
    output_variables=["query", "contexts", "text", "metadata"],
)

### Execute the Model

In [22]:
def format_mapped_vars(out, user_var):

    return pd.DataFrame(
        {
            "user_variable": user_var,
            "user_description": out["query"],
            "matched_text": out["metadata"]["matched_text"],
            "uid": out["metadata"]["uid"],
            "relevance_score": out["metadata"]["relevance_score"],
        }
    )

In [23]:
def format_llm_res_df(out, user_var):

    return pd.DataFrame(
        {
            "user_variable": user_var,
            "user_description": out["query"],
            "llm_response": out["text"]
        }
    )

In [24]:
def execute_rag_chain(df, mapped_vars_df, llm_res_df):
    total_length = len(df)
    print("Starting execution of RAG chain.")
    print(f"Total length of DataFrame: {total_length}")

    try:
        for index, row in df.iterrows():
            question = row[USER_SUBMITTED_LABEL_COL]
            out = rag_chain({"question": question})
            mapped_vars_df = pd.concat([mapped_vars_df, format_mapped_vars(out, row[USER_SUBMITTED_VARIABLE_COL])])
            llm_res_df = pd.concat([llm_res_df, format_llm_res_df(out, row[USER_SUBMITTED_VARIABLE_COL])])

            # Log progress at every 25% interval
            if index % (total_length // 4) == 0 and index != 0:
                print(f"Processed {index} rows, which is {(index/total_length)*100}% of the total.")
        print("Processing completed.")
        return mapped_vars_df, llm_res_df
    except Exception as e:
        print(f"An error occurred: {e}")
        return mapped_vars_df, llm_res_df

## Map to orginal data

In [25]:
mapped_vars_df = pd.DataFrame(columns=[
    "user_variable", # lowest
    "user_description", # out[query] i.e. middle
    "matched_text", # out[metadata][matched_text]
    "uid", # highest out[metadata][uid]
    "relevance_score",  # highest out[metadata][uid]
])


In [26]:
llm_res_df = pd.DataFrame(columns=[
    "user_variable", # lowest
    "user_description", # out[query] i.e. middle
    "llm_response", # out[text] i.e. middle
])


In [27]:
mapped_vars_df, llm_res_df = execute_rag_chain(df, mapped_vars_df, llm_res_df)

Starting execution of RAG chain.
Total length of DataFrame: 104


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/var/folders/mn/lqj94hqs2js64_82jtssv38h0000gn/T/ipykernel_5679/2056079758.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mapped_vars_df = pd.concat([mapped_vars_df, format_mapped_vars(out, row[USER_SUBMITTED_VARIABLE_COL])])
Query has been truncated from the right to 256 tokens from 1164 tokens.


An error occurred: You are using a Trial key, which is limited to 10 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.ai/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions


In [28]:
mapped_vars_df['uid'] = mapped_vars_df['uid'].astype(int)
mapped_vars_df.to_csv('llm_output/mapped_vars_df.csv', index=False)

mapped_vars_df.head()


,user_variable,user_description,uid,relevance_score
0,submitter_id,A project-specific identifier for a node. This...,242980,0.338842
1,submitter_id,A project-specific identifier for a node. This...,242989,0.266835
2,submitter_id,A project-specific identifier for a node. This...,242990,0.264358
0,participant_id,Unique identifier that can be used to retrieve...,177036,0.944074
1,participant_id,Unique identifier that can be used to retrieve...,177033,0.941331


In [29]:
llm_res_df.to_csv('llm_output/llm_res_df.csv', index=False)


llm_res_df.head()

,user_variable,user_description,llm_response
0,submitter_id,A project-specific identifier for a node. This...,"(lines, [1. UNIQUE COMMUNITY IDENTIFIER - This..."
0,participant_id,Unique identifier that can be used to retrieve...,"(lines, [Existing Variables:, 1. PARTICIPANT I..."
0,consent_codes,Data Use Restrictions that are used to indicat...,"(lines, [1. PERMISSION FOR NON-PROFIT USE OF D..."
0,amputation_type,"If amputated, the amputation type for leg, abo...","(lines, [Existing Variables:, 1. AMPUTATION OF..."
0,cohort_id,The study subgroup that the participant belong...,"(lines, [1. DONOR BLOOD GROUP - This existing ..."


In [30]:
llm_res_df


,user_variable,user_description,llm_response
0,submitter_id,A project-specific identifier for a node. This...,"(lines, [1. UNIQUE COMMUNITY IDENTIFIER - This..."
0,participant_id,Unique identifier that can be used to retrieve...,"(lines, [Existing Variables:, 1. PARTICIPANT I..."
0,consent_codes,Data Use Restrictions that are used to indicat...,"(lines, [1. PERMISSION FOR NON-PROFIT USE OF D..."
0,amputation_type,"If amputated, the amputation type for leg, abo...","(lines, [Existing Variables:, 1. AMPUTATION OF..."
0,cohort_id,The study subgroup that the participant belong...,"(lines, [1. DONOR BLOOD GROUP - This existing ..."
0,data_submission_guid,A Globally Unique IDentifier (GUID) provided b...,"(lines, [1. UNIQUE SUBJECT IDENTIFIER - This v..."
0,derived_parent_subject_id,Unique subject identifier associated with pare...,"(lines, [1. UNIQUE SUBJECT IDENTIFIER - This v..."
0,derived_topmed_subject_id,Unique subject identifier associated with TOPM...,"(lines, [Existing Variables:, 1. TOPMED ACCESS..."
0,eligibility,A verification field of whether the Donor has ...,"(lines, [Existing Variables:, 1. ELIGIBILITY V..."
0,geographic_site,"""Site where the subject was recruted. ARIC_A=f...","(lines, [Existing Variables:, 1. ARIC SURVEILL..."


In [31]:
mapped_vars_df


,user_variable,user_description,uid,relevance_score
0,submitter_id,A project-specific identifier for a node. This...,242980,0.338842
1,submitter_id,A project-specific identifier for a node. This...,242989,0.266835
2,submitter_id,A project-specific identifier for a node. This...,242990,0.264358
0,participant_id,Unique identifier that can be used to retrieve...,177036,0.944074
1,participant_id,Unique identifier that can be used to retrieve...,177033,0.941331
2,participant_id,Unique identifier that can be used to retrieve...,177035,0.851336
0,consent_codes,Data Use Restrictions that are used to indicat...,180823,0.980431
1,consent_codes,Data Use Restrictions that are used to indicat...,21660,0.979551
2,consent_codes,Data Use Restrictions that are used to indicat...,180821,0.979235
0,amputation_type,"If amputated, the amputation type for leg, abo...",4733,0.885632


### Map to original variables

In [32]:
import pandas as pd

tsv_file_path = './lookup_table/biolincc_dedupLUT 1.tsv'
df = pd.read_csv(tsv_file_path, delimiter='\t')

df.head()


,Study,Variable,Label,UID
0,AADR,AUGDATE_FZD,"AUGMENTATION DATE, FZD",25106
1,AADR,CENTER,CENTER WHERE TESTING DONE,44791
2,AADR,CLINIC,CLINIC,48834
3,AADR,CLINIC,CLINICAL CENTER,48894
4,AADR,CLINIC,CLINICAL CENTER CODE,48895


In [33]:
242989
df[df['UID'] == 52785]

mapped_vars_df

,user_variable,user_description,uid,relevance_score
0,submitter_id,A project-specific identifier for a node. This...,242980,0.338842
1,submitter_id,A project-specific identifier for a node. This...,242989,0.266835
2,submitter_id,A project-specific identifier for a node. This...,242990,0.264358
0,participant_id,Unique identifier that can be used to retrieve...,177036,0.944074
1,participant_id,Unique identifier that can be used to retrieve...,177033,0.941331
2,participant_id,Unique identifier that can be used to retrieve...,177035,0.851336
0,consent_codes,Data Use Restrictions that are used to indicat...,180823,0.980431
1,consent_codes,Data Use Restrictions that are used to indicat...,21660,0.979551
2,consent_codes,Data Use Restrictions that are used to indicat...,180821,0.979235
0,amputation_type,"If amputated, the amputation type for leg, abo...",4733,0.885632


In [34]:
for index, row in mapped_vars_df.iterrows():
    uid = row['uid']
    print(f'--- UID: {uid} --  INDEX: {index} --------------------------------------------------------------')
    matching_rows = df[df['UID'] == uid]

    related_vars = []
    for _, matching_row in matching_rows.iterrows():

        related_vars.append({
            'Study': matching_row['Study'].strip(),
            'Variable': matching_row['Variable'].strip(),
            'Label': matching_row['Label'].strip(),
            'UID': matching_row['UID']
        })
    
    print(related_vars)
    mapped_vars_df.at[index, 'related_vars'] = related_vars
    # try:
    #     mapped_vars_df.at[index, 'related_vars'] = str(related_vars)
    # except:
    #     mapped_vars_df.at[index, 'related_vars'] = []


mapped_vars_df

--- UID: 242980 --  INDEX: 0 --------------------------------------------------------------
[{'Study': 'HCS', 'Variable': 'COMMUNITYID', 'Label': 'UNIQUE COMMUNITY IDENTIFIER', 'UID': 242980}]


ValueError: Must have equal len keys and value when setting with an iterable

In [ ]:
mapped_vars_df